In [1]:
COLAB = False
if COLAB:
    from google.colab import drive
    drive.mount('/content/gdrive')
    DRIVE_PATH = F'/content/gdrive/My Drive/evaluation-pipeline/' # Path to data on Google Drive
    REPO_PATH = F'/content/evaluation-pipeline/'
else:
    DRIVE_PATH = F'' # Local path
    REPO_PATH = ''


#import importlib.util
%load_ext autoreload
%autoreload 2
%matplotlib inline

In [2]:
if COLAB:
    !git clone -b development https://github.com/lovis-heindrich/evaluation-pipeline.git

Cloning into 'evaluation-pipeline'...
remote: Enumerating objects: 3, done.
remote: Counting objects: 100% (3/3), done.
remote: Compressing objects: 100% (3/3), done.
remote: Total 538 (delta 0), reused 1 (delta 0), pack-reused 535
Receiving objects: 100% (538/538), 204.15 MiB | 13.16 MiB/s, done.
Resolving deltas: 100% (337/337), done.


In [ ]:
if COLAB:
    %cd /content/evaluation-pipeline
    !git pull

/content/evaluation-pipeline
Already up to date.


In [3]:
# Imports
import torch
import sys
import os
import pandas as pd
from torch import nn
from PIL import Image
import numpy as np

# Import local files from github
sys.path.append(REPO_PATH+"src")
import models
import train
import utils

from IPython.display import display, HTML

torch.manual_seed(0)
torch.backends.cudnn.deterministic = True
torch.backends.cudnn.benchmark = False
np.random.seed(0)

In [4]:
# Parameters
load_model = False # Load old model instead of training
save_model = False # Overwrite old model after training

# Image parameters
num_channels = 3
image_size = 64
num_classes = 14

# Training parameters
batch_size = 128
num_epochs = 5

# GPU
use_gpu = True
device = torch.device("cuda:0" if torch.cuda.is_available() and use_gpu else "cpu")
print(device)

cuda:0


In [5]:
# Init cure tsd dataset
import cure_tsd_loader
path = DRIVE_PATH+"Data/cure_tsd_64x64.npz"
trainloader, testloader, trainset, testset, data = cure_tsd_loader.get_dataloader(path, 0.8, 128)

In [6]:
# Init classifiers
def getBaselineClassifier():
    classifier_baseline = models.BaseClassifier(num_channels, num_classes)
    classifier_baseline.to(device)
    c_loss = nn.CrossEntropyLoss()
    c_baseline_optimizer = torch.optim.Adam(classifier_baseline.parameters(), 0.0002)
    baseline_acc = train.train(classifier_baseline, c_baseline_optimizer, c_loss, 4, trainloader, testloader, device, log=False)
    return classifier_baseline

def getMCClassifier():
    classifier_mc = models.MCSampleClassifier(num_channels, num_classes, 0.6, device)
    classifier_mc.to(device)
    c_loss = nn.CrossEntropyLoss()
    c_mc_optimizer = torch.optim.Adam(classifier_mc.parameters(), 0.0002)
    mc_acc = train.train(classifier_mc, c_mc_optimizer, c_loss, 8, trainloader, testloader, device, log=False)
    return classifier_mc

In [ ]:
if not COLAB:
    if os.path.isfile(DRIVE_PATH+"Data/models/classifier_baseline.pt") and os.path.isfile(DRIVE_PATH+"Data/models/classifier_mc.pt"):
        classifier_baseline = models.BaseClassifier(num_channels, num_classes)
        classifier_mc = models.MCSampleClassifier(num_channels, num_classes, 0.6, device)
        classifier_baseline.load_state_dict(torch.load(DRIVE_PATH+"Data/models/classifier_baseline.pt"))
        classifier_mc.load_state_dict(torch.load(DRIVE_PATH+"Data/models/classifier_mc.pt"))
        classifier_baseline.to(device)
        classifier_mc.to(device)
        print("Loaded")
    else:
        print("Baseline")
        classifier_baseline = getBaselineClassifier()
        print("\nMC Dropout")
        classifier_mc = getMCClassifier()
        torch.save(classifier_baseline.state_dict(), DRIVE_PATH+"Data/models/classifier_baseline.pt")
        torch.save(classifier_mc.state_dict(), DRIVE_PATH+"Data/models/classifier_mc.pt")

#print(train.test(classifier_baseline, testloader, device))
#print(train.test(classifier_mc, testloader, device))

In [7]:
from image_transforms import gen_crop_transform, gen_noise_transform, gen_grey_transform, gen_snow_transform, gen_rain_transforms, gen_fog_transform, gen_perspective_transform, gen_blur_transform, gen_rotate_transform, gen_reflection_transform
import classification

pipeline = classification.ClassificationPipeline(device, testset, batch_size)
print("Test set size:", len(testset))

Test set size: 1792


/usr/local/lib/python3.6/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
noise_transform = [gen_noise_transform(0.2),
                   gen_noise_transform(0.35),
                   gen_noise_transform(0.4),
                   gen_noise_transform(0.45),
                   gen_noise_transform(0.5)]

grey_transform = [gen_grey_transform(0.2),
                  gen_grey_transform(0.4),
                  gen_grey_transform(0.6),
                  gen_grey_transform(0.8),
                  gen_grey_transform(1)]
                
snow_path = REPO_PATH+"res/snow1.jpg"
snow_img = Image.open(snow_path)
snow_transform = [gen_snow_transform([1], snow_img),
                  gen_snow_transform([1, 1], snow_img),
                  gen_snow_transform([1.5, 1.5], snow_img),
                  gen_snow_transform([1.5, 1.5, 1], snow_img),
                  gen_snow_transform([1.5, 1.5, 1.5], snow_img)]

rain_path = REPO_PATH+"res/rain3.jpg"
rain_img = Image.open(rain_path)
rain_transform = [gen_rain_transforms([0.1, 0.1], 30, rain_img),
                   gen_rain_transforms([0.2, 0.2], 30, rain_img),
                   gen_rain_transforms([0.3, 0.3], 30, rain_img),
                   gen_rain_transforms([0.4, 0.4], 30, rain_img),
                   gen_rain_transforms([0.5, 0.4, 0.2], 30, rain_img)]

fog_transform = [gen_fog_transform(0.3),
                 gen_fog_transform(0.4),
                 gen_fog_transform(0.5),
                 gen_fog_transform(0.6),
                 gen_fog_transform(0.7),]
        
perspective_transform = [gen_perspective_transform(0.2, 0.9),
                         gen_perspective_transform(0.3, 0.85),
                         gen_perspective_transform(0.4, 0.8),
                         gen_perspective_transform(0.5, 0.75),
                         gen_perspective_transform(0.6, 0.7)]
                    
blur_transform = [gen_blur_transform(1.5), 
                  gen_blur_transform(2), 
                  gen_blur_transform(2.5), 
                  gen_blur_transform(3),
                  gen_blur_transform(3.5)]
                
rotation_transform = [gen_rotate_transform(10),
                       gen_rotate_transform(15), 
                       gen_rotate_transform(20), 
                       gen_rotate_transform(25), 
                       gen_rotate_transform(30)]

crop_transform = [gen_crop_transform(62),
                       gen_crop_transform(60), 
                       gen_crop_transform(58), 
                       gen_crop_transform(56), 
                       gen_crop_transform(54)]
                    
reflection_transform = [gen_reflection_transform(8, 1),
                        gen_reflection_transform(12, 1),
                        gen_reflection_transform(16, 1),
                        gen_reflection_transform(20, 1),
                        gen_reflection_transform(24, 1)]

In [ ]:
def store_imgs():
    # Print some test images
    snow_print_img = pipeline.get_test_images(1, snow_transform)[4]
    reflection_print_img = pipeline.get_test_images(1, reflection_transform)[4]
    fog_print_img = pipeline.get_test_images(1, fog_transform)[4]
    rotate_print_img = pipeline.get_test_images(1, rotation_transform)[4]
    imgs = [snow_print_img, reflection_print_img, fog_print_img, rotate_print_img]
    full_tensor = torch.cat(imgs)
    #torchvision.utils.save_image(full_tensor, DRIVE_PATH+"Results/classification_examples_compact.png")

import torchvision 
def show_imgs(store=True):
    # Print some test images
    noise_img = pipeline.get_test_images(8, noise_transform)
    grey_img = pipeline.get_test_images(8, grey_transform)
    snow_img = pipeline.get_test_images(8, snow_transform)
    rain_img = pipeline.get_test_images(8, rain_transform)
    fog_img = pipeline.get_test_images(8, fog_transform)
    perspective_img = pipeline.get_test_images(8, perspective_transform)
    blur_img = pipeline.get_test_images(8, blur_transform)
    rotation_img = pipeline.get_test_images(8, rotation_transform)
    crop_img = pipeline.get_test_images(8, crop_transform)
    reflection_img = pipeline.get_test_images(8, reflection_transform)

    #noise
    torchvision.utils.save_image(torchvision.utils.make_grid(torch.cat(noise_img), 8), DRIVE_PATH + "Images_thesis/transformations/noise_all.png")
    torchvision.utils.save_image(torchvision.utils.make_grid(noise_img[2], 8), DRIVE_PATH + "Images_thesis/transformations/noise_l3.png")
    #grey
    torchvision.utils.save_image(torchvision.utils.make_grid(torch.cat(grey_img), 8), DRIVE_PATH + "Images_thesis/transformations/grey_all.png")
    torchvision.utils.save_image(torchvision.utils.make_grid(grey_img[2], 8), DRIVE_PATH + "Images_thesis/transformations/grey_l3.png")
    #snow
    torchvision.utils.save_image(torchvision.utils.make_grid(torch.cat(snow_img), 8), DRIVE_PATH + "Images_thesis/transformations/snow_all.png")
    torchvision.utils.save_image(torchvision.utils.make_grid(snow_img[2], 8), DRIVE_PATH + "Images_thesis/transformations/snow_l3.png")
    #rain
    torchvision.utils.save_image(torchvision.utils.make_grid(torch.cat(rain_img), 8), DRIVE_PATH + "Images_thesis/transformations/rain_all.png")
    torchvision.utils.save_image(torchvision.utils.make_grid(rain_img[2], 8), DRIVE_PATH + "Images_thesis/transformations/rain_l3.png")
    #fog
    torchvision.utils.save_image(torchvision.utils.make_grid(torch.cat(fog_img), 8), DRIVE_PATH + "Images_thesis/transformations/fog_all.png")
    torchvision.utils.save_image(torchvision.utils.make_grid(fog_img[2], 8), DRIVE_PATH + "Images_thesis/transformations/fog_l3.png")
    #perspective
    torchvision.utils.save_image(torchvision.utils.make_grid(torch.cat(perspective_img), 8), DRIVE_PATH + "Images_thesis/transformations/perspective_all.png")
    torchvision.utils.save_image(torchvision.utils.make_grid(perspective_img[2], 8), DRIVE_PATH + "Images_thesis/transformations/perspective_l3.png")
    #blur
    torchvision.utils.save_image(torchvision.utils.make_grid(torch.cat(blur_img), 8), DRIVE_PATH + "Images_thesis/transformations/blur_all.png")
    torchvision.utils.save_image(torchvision.utils.make_grid(blur_img[2], 8), DRIVE_PATH + "Images_thesis/transformations/blur_l3.png")
    #rotation
    torchvision.utils.save_image(torchvision.utils.make_grid(torch.cat(rotation_img), 8), DRIVE_PATH + "Images_thesis/transformations/rotation_all.png")
    torchvision.utils.save_image(torchvision.utils.make_grid(rotation_img[2], 8), DRIVE_PATH + "Images_thesis/transformations/rotation_l3.png")
    #crop
    torchvision.utils.save_image(torchvision.utils.make_grid(torch.cat(crop_img), 8), DRIVE_PATH + "Images_thesis/transformations/crop_all.png")
    torchvision.utils.save_image(torchvision.utils.make_grid(crop_img[2], 8), DRIVE_PATH + "Images_thesis/transformations/crop_l3.png")
    #reflection
    torchvision.utils.save_image(torchvision.utils.make_grid(torch.cat(reflection_img), 8), DRIVE_PATH + "Images_thesis/transformations/reflection_all.png")
    torchvision.utils.save_image(torchvision.utils.make_grid(reflection_img[2], 8), DRIVE_PATH + "Images_thesis/transformations/reflection_l3.png")



#show_imgs()
#store_imgs()

In [ ]:
 # Manual test
#r = pipeline.run_test(classifier_baseline, noise_transform)#grey_transform)

In [8]:
def full_run(name, transform, showimage=False, n=10):
    # Show challenge images
    print(name)
    if showimage:
        pipeline.get_test_images(4, transform)

    # Run baseline evaluation
    print("\nBaseline")
    baseline, baseline_df = (pipeline.average_run_tests(n, transform, getBaselineClassifier))

    # Run MC dropout evaluation
    print("MC Dropout")
    mc, mc_df = (pipeline.average_run_tests(n, transform, getMCClassifier))
    print("")

    # Create output df
    mc.columns = ["mean_mc", "std_mc"]
    df = baseline.merge(mc, left_index=True, right_index=True)
    index_arr = [["Baseline", "Baseline", "MC Dropout", "MC Dropout"],["Mean", "Std", "Mean", "Std"]]
    index_tpl = list(zip(*index_arr))
    index = pd.MultiIndex.from_tuples(index_tpl, names=["Classifier", "Measure"])
    df.columns = index
    df = df.round(4)
    if showimage:
        display(df)
    return df, [baseline_df, mc_df]

In [9]:
def get_raw_data(df, classifier, transformation, measure):
    df = df.unstack().reset_index().drop(["level_1"], axis = 1)
    df["Classifier"] = classifier
    df["Transformation"] = transformation
    df.rename(columns={0: measure}, inplace=True)
    return df

def combine_raw_dfs_old(dfs, transformation, measures=["Accuracy", "ROC", "Brier"]):
    res = []
    for measure in measures:
        df1 = get_raw_data(dfs[0][measure], "Baseline", transformation, measure)
        df2 = get_raw_data(dfs[1][measure], "MC Dropout", transformation, measure)
        res.append(pd.concat([df1, df2], axis=0))
    return res

def combine_raw_dfs(dfs, transformation, measures=["Accuracy", "ROC", "Brier"]):
    measure = measures[0]
    df1 = get_raw_data(dfs[0][measure], "Baseline", transformation, measure)
    df2 = get_raw_data(dfs[1][measure], "MC Dropout", transformation, measure)
    res = pd.concat([df1, df2], axis=0)
    if len(measures) > 1:
        for measure in measures[1:]:
            df1 = get_raw_data(dfs[0][measure], "Baseline", transformation, measure)
            df2 = get_raw_data(dfs[1][measure], "MC Dropout", transformation, measure)
            combined = pd.concat([df1, df2], axis=0)
            res[measure] = combined[measure]
    return res

In [10]:
def raw_data_run(transform_name, transform, df_name, save_name):
    res, dfs = full_run(transform_name, transform, n=20)

    df = combine_raw_dfs(dfs, df_name)
    res_path = DRIVE_PATH + 'Results/Classification_' + save_name + '_results_final2.p'
    utils.save_results((res, df), res_path)

In [13]:
from torchvision import transforms
#base_transform = [transforms.Compose([transforms.ToPILImage(),
 #   transforms.ToTensor()])]*5

base_transform = [lambda x: x]*5
base_res, dfs = full_run("No transform", base_transform, n=200)

res = base_res

base_df = combine_raw_dfs(dfs, "No transform")

res_path = DRIVE_PATH + 'Results/Classification_base_results_final_big.p'
utils.save_results((base_res, base_df), res_path)
base_df.to_csv(DRIVE_PATH + 'Results/Classification_base_results_final_big.csv')

No transform

Baseline



MC Dropout


In [ ]:
noise_res, dfs = full_run("Noise transform", noise_transform, n=20)
noise_df = combine_raw_dfs(dfs, "Noise")
res_path = DRIVE_PATH + 'Results/Classification_noise_results_final2.p'
utils.save_results((noise_res, noise_df), res_path)

Noise transform

Baseline



MC Dropout


In [ ]:
grey_res, dfs = full_run("Grey transform", grey_transform, n=20)

grey_df = combine_raw_dfs(dfs, "Grey")
res_path = DRIVE_PATH + 'Results/Classification_grey_results_final2.p'
utils.save_results((grey_res, grey_df), res_path)

Grey transform

Baseline



MC Dropout


In [ ]:
#snow_res = full_run("Snow transform", snow_transform, n=10)
raw_data_run("Snow transform", snow_transform, "Snow", "snow")

In [ ]:
res = (noise_res, grey_res, snow_res)
res_path = DRIVE_PATH + 'Results/Classification_results_noise_grey_snow_final.p'
utils.save_results(res, res_path)

In [ ]:
#rain_res = full_run("Rain transform", rain_transform, n=10)
raw_data_run("Rain transform", rain_transform, "Rain", "rain")

Rain transform

Baseline



MC Dropout


In [ ]:
#fog_res = full_run("Fog transform", fog_transform, n=10)
raw_data_run("Fog transform", fog_transform, "Fog", "fog")

Fog transform

Baseline



MC Dropout


In [ ]:
#perspective_res = full_run("Perspective transform", perspective_transform, n=10)
raw_data_run("Perspective transform", perspective_transform, "Perspective", "perspective")

Perspective transform

Baseline



MC Dropout


In [ ]:
#blur_res = full_run("Blur transform", blur_transform, n=10)
raw_data_run("Blur transform", blur_transform, "Blur", "blur")

Blur transform

Baseline



MC Dropout


In [ ]:
#rotation_res = full_run("Rotation transform", rotation_transform, n=10)
raw_data_run("Rotation transform", rotation_transform, "Rotation", "rotation")

Rotation transform

Baseline



MC Dropout


In [ ]:
#crop_res = full_run("Crop transform", crop_transform, n=10)
raw_data_run("Crop transform", crop_transform, "Crop", "crop")

Crop transform

Baseline



MC Dropout


In [ ]:
#reflection_res = full_run("Reflection transform", reflection_transform, n=10)
raw_data_run("Reflection transform", reflection_transform, "Reflection", "reflection")

Reflection transform

Baseline



MC Dropout


In [ ]:
res = (noise_res, grey_res, snow_res, rain_res, fog_res, perspective_res, blur_res, rotation_res, crop_res, reflection_res)
res_path = DRIVE_PATH + 'Results/Classification_results_final.p'
utils.save_results(res, res_path)